In [46]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l 

In [47]:
#生成数据集
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = d2l.synthetic_data(true_w,true_b,1000)

In [48]:
#读取数据集
def load_array(data_arrays,batch_size,is_train = True):    #@save
    """"构建一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

torch.utils.data.DataLoader的用法:
class torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)

dataset (Dataset): 封装后的数据集。
batch_size (python:int,optional): 每一批加载的样本量，默认值为1。
shuffle (bool,optional): 设置为True时,每一个epoch重新打乱数据顺序。
sampler (Sampler,optional): 定义在数据集中进行采样的策略，如果被指定，则False必须为shuffle。
batch_sampler (Sampler,optional): 类似sampler，但是一次返回一批索引。互斥有batch_size，shuffle，sampler和drop_last。
num_workers (python:int,optional): 多少个子进程用于数据加载。0表示将在主进程中加载数据,默认值为0。
collate_fn(callable,optional): 合并样本列表以形成张量的小批量。在从地图样式数据集中使用批量加载时使用。
pin_memory (bool,optional): 如果为True，则数据加载器在将张量返回之前将其复制到CUDA固定的内存中。
drop_last (bool,optional): 设置为True，如果数据集大小不能被该批次大小整除则删除最后一个不完整的批次。如果False，数据集的大小不能被批量大小整除，那么最后一个批量将更小，默认值为False。
timeout (numeric,optional): 如果为正，则为从worker收集批次的超时值。应始终为非负数,默认值为0。
worker_init_fn (callable,optional): 如果不是None，则在种子工作之后和数据加载之前，将在每个工作程序子进程上调用此程序，并以工作程序ID作为输入,取值为[0, num_workers - 1]或None。


In [49]:
batch_size = 10
data_iter = load_array((features,labels),batch_size)
next(iter(data_iter))

[tensor([[-0.4783, -0.0118],
         [ 0.0746,  0.5166],
         [ 3.0961,  0.0916],
         [ 1.2659, -0.0085],
         [ 0.6784,  0.9930],
         [-0.8871,  0.6635],
         [-0.2796, -0.2754],
         [-0.5801, -1.5430],
         [ 0.5988, -1.3262],
         [ 0.7577, -0.0182]]),
 tensor([[ 3.2768],
         [ 2.5748],
         [10.0720],
         [ 6.7551],
         [ 2.1687],
         [ 0.1803],
         [ 4.5662],
         [ 8.3120],
         [ 9.9163],
         [ 5.7615]])]

In [50]:
#定义模型
#nn为神经网络的缩写

from torch import nn
net = nn.Sequential(nn.Linear(2,1))
print(net)



Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


Sequential()模型只有一组输入和一组输出。各层之间按照先后顺序进行堆叠。前面一层的输出就是后面一次的输入。通过不同层的堆叠，构建出神经网络。
torch.nn.Linear(in_features, out_features, bias=True) 函数是一个线性变换函数：

其中，in_features为输入样本的大小，out_features为输出样本的大小，bias默认为true。如果设置bias = false那么该层将不会学习一个加性偏差


In [51]:
#初始化参数模型
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [52]:
#定义损失函数
loss = nn.MSELoss()

很多的loss函数都有size_average和reduce两个布尔类型的参数，因为一般损失函数都是直接计算batch的数据，因此返回的loss结果都是维度为(batch_size,)的向量。

1、如果reduce=False,那么size_average参数失效，直接返回向量形式的loss

2、如果redcue=true,那么loss返回的是标量。

   2.a: if size_average=True, 返回loss.mean();#就是平均数

   2.b: if size_average=False,返回loss.sum()

注意：默认情况下，reduce=true,size_average=true


In [53]:
#定义优化算法
trainer = torch.optim.SGD(net.parameters(),lr=0.03)
print(net.parameters)

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)>


torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9,weight_decay=wd)
第一个参数包括权重w，和偏置b等是神经网络中的参数，也是SGD优化的重点
第二个参数lr是学习率。sgd中的学习率lr的作用可以理解为：p ′ = p − lr ∗ dp 其中p就是模型中的参数比如：权重(w), 偏置(b)。
dp就是对p的一阶求导，lr即学习率，p′ 为p的另一种形式，即用来替换上一次的p
weight_decay用来防止过拟合

In [54]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch{epoch + 1}, loss{l:f}')

epoch1, loss0.000289
epoch2, loss0.000103
epoch3, loss0.000102


In [55]:
w = net[0].weight.data
print('w的估计误差:',true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差:',true_b - b)

w的估计误差: tensor([ 0.0003, -0.0009])
b的估计误差: tensor([0.0002])
